<a href="https://colab.research.google.com/github/pashkev14/DD_CL-NLP_Final/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Импортировали NLTK и загрузили необходимую модель для TextBlob.

In [2]:
from textblob import TextBlob # для анализа тональности будем использовать TextBlob

In [3]:
# функция вычисления полярности текста
def sentiment_score(review):
  blob = TextBlob(review)
  return blob.sentiment.polarity

In [4]:
# функция оценки полярности на основе вычисленного результата
def sentiment_grade(score):
  # будем считать, что полярность меньше -0.05 - отрицательная,
  # больше 0.05 - положительная,
  # все остальное будем считать нейтральным
  if score < -0.05:
    return 'отрицательный'
  if score > 0.05:
    return 'положительный'
  return 'нейтральный'

In [5]:
# функция вычисления субъективности текста
def subjectivity_score(review):
  blob = TextBlob(review)
  return blob.sentiment.subjectivity

In [6]:
# функция оценки субъективности на основе вычисленного результата
def subjectivity_grade(score):
  # будем считать, что оценка от 0.0 до 0.3 - объективная,
  # от 0.3 до 0.7 - нейтральная,
  # от 0.7 до 1.0 - субъективная
  if score < 0.3:
    return 'объективный'
  if score < 0.7:
    return 'нейтральный'
  return 'субъективный'

In [7]:
import pandas as pd # для чтения xlsx файла с отзывами и записи в xlsx файл результатов будем использовать pandas

# читаем файл и применяем к нему наши функции сентимент-анализа
data = pd.read_excel('imdb_reviews.xlsx', sheet_name='Sheet1')
data['polarity_score'] = data['review_text'].apply(sentiment_score)
data['polarity_grade'] = data['polarity_score'].apply(sentiment_grade)
data['subjectivity_score'] = data['review_text'].apply(subjectivity_score)
data['subjectivity_grade'] = data['subjectivity_score'].apply(subjectivity_grade)
# записываем данные в файл
with pd.ExcelWriter('imdb_reviews_analysis.xlsx') as writer:
  data.to_excel(writer, sheet_name='General')
  data[data['polarity_grade'] == 'отрицательный'].to_excel(writer, sheet_name='NegativePolarity')
  data[data['polarity_grade'] == 'нейтральный'].to_excel(writer, sheet_name='NeutralPolarity')
  data[data['polarity_grade'] == 'положительный'].to_excel(writer, sheet_name='PositivePolarity')
  data[data['subjectivity_grade'] == 'объективный'].to_excel(writer, sheet_name='Objective')
  data[data['subjectivity_grade'] == 'нейтральный'].to_excel(writer, sheet_name='Neutral')
  data[data['subjectivity_grade'] == 'субъективный'].to_excel(writer, sheet_name='Subjective')

Результаты очень интересные:
1. В категорию положительных отзывов попали даже отрицательные отзывы (даже не с пограничной оценкой, а с явно выраженной положительной оценкой)
2. Большинство отрицательных отзывов получили нейтральную оценку
3. Положительные отзывы были оценены верно и попали в категорию положительных отзывов
4. Ни один отзыв не был оценен как объективный, все отзывы имеют как эмоции, так и факты
5. Но и сильно субъективным отзывом был оценен только один экземпляр - следовательно, из выборки все отзывы стремятся к сочетанию эмоциональности и фактов